In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.feature_selection import SelectKBest, f_classif
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats

In [24]:
data = pd.read_csv('sampled_tbc_training_data.csv')

In [25]:
print(data.isnull().sum())

Customer_ID    0
recency        0
frequency      0
value          0
age            0
              ..
ITADMILLNM     0
ITADSEPLUS     0
ITBCCORE       0
ITOYODRVN      0
OREXCEPT       0
Length: 229, dtype: int64


In [26]:
duplicates  = data.duplicated()
data = data[~duplicates]

In [27]:
data = data.iloc[:, ~data.columns.duplicated()]

In [28]:
y = data['churn']
x = data.drop('churn', axis=1)

In [29]:
for col in x.columns:
    if x[col].nunique() == 2:
        x[col] = x[col].astype('category')

In [30]:
cat_cols = x.select_dtypes(include=['category']).columns
num_cols = x.select_dtypes(exclude=['category']).columns

In [31]:
scaler = StandardScaler()
x[num_cols] = scaler.fit_transform(x[num_cols])

In [32]:

anova_results = {}
for col in num_cols:
    group1 = x[y == 0][col]
    group2 = x[y == 1][col]
    f_stat, p_value = stats.f_oneway(group1, group2)
    anova_results[col] = p_value < 0.05
selected_num_cols = [col for col, significant in anova_results.items() if significant]

chi2_results = {}
for col in cat_cols:
    contingency_table = pd.crosstab(x[col], y)
    chi2, p_value, _, _ = stats.chi2_contingency(contingency_table)
    chi2_results[col] = p_value < 0.05
selected_cat_cols = [col for col, significant in chi2_results.items() if significant]



d:\AI COE Trainings\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:579: ConstantInputWarning: Each of the input arrays is constant; the F statistic is not defined or infinite
  res = hypotest_fun_out(*samples, **kwds)


In [33]:
len(selected_num_cols), len(selected_cat_cols)

(73, 6)

In [34]:
selected_features = selected_num_cols + selected_cat_cols
x = x[selected_features]

In [35]:
x.shape

(2000, 79)

In [36]:
#logistic regression model
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
model = LogisticRegression(max_iter=1000)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("f1-score:", 2 * (precision_score(y_test, y_pred) * recall_score(y_test, y_pred)) / (precision_score(y_test, y_pred) + recall_score(y_test, y_pred)))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.8425
Precision: 0.8157894736842105
Recall: 0.3563218390804598
f1-score: 0.496
Confusion Matrix:
 [[306   7]
 [ 56  31]]


In [41]:
# Plot feature importance
plt.figure(figsize=(12, 6))
xgb.plot_importance(xgb_model, max_num_features=20)
plt.title('Top 20 Most Important Features')
plt.tight_layout()
plt.show()

# Plot ROC curves to compare models
from sklearn.metrics import roc_curve

# Get predictions for both models
lr_proba = model.predict_proba(x_test)[:,1]
xgb_proba = xgb_model.predict_proba(x_test)[:,1]

# Calculate ROC curves
lr_fpr, lr_tpr, _ = roc_curve(y_test, lr_proba)
xgb_fpr, xgb_tpr, _ = roc_curve(y_test, xgb_proba)

# Plot ROC curves
plt.figure(figsize=(10, 6))
plt.plot(lr_fpr, lr_tpr, label=f'Logistic Regression (AUC = {roc_auc_score(y_test, lr_proba):.3f})')
plt.plot(xgb_fpr, xgb_tpr, label=f'XGBoost (AUC = {roc_auc_score(y_test, xgb_proba):.3f})')
plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves Comparison')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

<Figure size 1200x600 with 0 Axes>